In [1]:
import json
import random
import re
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Arrow 파일 경로 설정
train_file_path = os.path.join(parent_dir, 'data', 'train_dataset')
test_file_path = os.path.join(parent_dir, 'data', 'test_dataset')
dataset = load_from_disk(train_file_path)
test_dataset=load_from_disk(test_file_path)

wiki_path = os.path.join(parent_dir, 'data', "wikipedia_documents.json")

with open(wiki_path, "r", encoding="utf-8") as json_file:
    json_data = json.load(json_file)


df_wiki=pd.read_json(wiki_path).transpose()

C:\Users\spdlq\.conda\envs\boost-camp1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
test_dataset=test_dataset

train_df=pd.DataFrame(train_dataset)
valid_df=pd.DataFrame(valid_dataset)
test_df=pd.DataFrame(test_dataset)

print(f"Train length : {len(train_df)}")
print(f"Valid length : {len(valid_df)}")
print(f"Test length : {len(test_df)}")
print(f"df_wiki length : {len(df_wiki)}")

Train length : 3952
Valid length : 240
Test length : 600
df_wiki length : 60613


,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [55]:
def generate_import_word(text):
    try:
        # 프롬프트 수정: 핵심 단어를 추출하도록 요청
        prompt = (f"Context: {text}\n"
                  f"위 Context에서 핵심 질문의 정답으로 쓰일 수 있는 단어가 뭐가 있을까?\n"
                  f"MRC Task이고, 질문은 사실기반 질문/이유 질문/비교질문 일 수 있어.\n"
                  f"접두사나 접미사 붙이지 말고, 정답 단어나 키워드를 하나만 알려줘.")
        
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=100,  # 단어 추출을 위한 적당한 토큰 수
            temperature=0.5,
        )
        
        # 응답 처리 부분: 응답에서 핵심 단어 추출
        response_text = completion.choices[0].message.content.strip()
        keywords = response_text.split("\n")  # 여러 단어가 반환되면 줄바꿈 기준으로 분리
        
    except Exception as e:
        print(f"Error: {e}")
        keywords = []  # 에러 발생 시 빈 리스트 반환
        
    return keywords

In [62]:
def generate_question(text, keyword):
    try:
        # 프롬프트 수정: 키워드와 문맥에 맞는 질문을 생성하도록 요청
        prompt = (f"Context: {text}\n"
                  f"위 Context에서 '{keyword}'를 정답으로 하는 질문을 만들어줘."
                  f"MRC Task의 데이터로 사용되기 때문에, Extraction-based MRC모델이 context와 질문을 통해서 {keyword}를 유추할 수 있어야해"
                  f"예시는 아래와 같아."
                  f"context : 《우리들의》는 한 러시아인 가족의 이야기다. 작품 속 화자는 자신의 가족 한 사람 한 사람에 대한 개성 넘치는 이야기들을 들려주고 있다. 《우리들의》에는 열세 명의 가족을 주인공으로 한, 열세 편의 이야기가 들어 있다."
                  f"keyword : 러시아인"
                  f"일때 만들어진 질문은 《우리들의》에서 다루고 있는 가정은 어느 나라 사람들인가?"
                  f"질문만 만들어서 간결하게 출력해줘")
        
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=100,  # 적절한 질문을 위해 적당한 토큰 수
            temperature=0.5,
        )
        
        # 응답 처리 부분: 생성된 질문 추출
        question = completion.choices[0].message.content.strip()
        
    except Exception as e:
        print(f"Error: {e}")
        question = None  # 에러 발생 시 None 반환
        
    return question



In [92]:
import pandas as pd
import os

# 파일 경로 설정
csv_file_path = "mrc_keywords_questions_data.csv"

# 빈 리스트 생성: 저장할 데이터를 담기 위한 리스트
data = []

# 테스트 실행 예시
df_wiki = df_wiki.copy()

# 미리 인덱스를 셔플한 리스트를 생성
shuffled_indices = list(df_wiki.index)
random.shuffle(shuffled_indices)
iter = 1000

# 기존 파일이 있으면 읽어오기
if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    print("Existing file loaded.")
else:
    df_existing = pd.DataFrame(columns=["title", "context", "document_id", "keyword", "start_pos", "question"])
    print("No existing file found, creating a new one.")

# 100개의 랜덤 텍스트에서 중요한 키워드 찾기 및 질문 생성
for i in range(iter):
    picked = df_wiki.loc[shuffled_indices[i], ['text', 'title', 'document_id']]
    
    # text 내에서 중요한 키워드 찾기
    keywords = generate_import_word(picked['text'])
    
    # 키워드와 text에 맞는 질문 생성
    if keywords:
        for keyword in keywords:
            # 키워드 시작위치 계산
            start_pos = picked['text'].find(keyword)
            
            # 키워드가 text에 존재하는지 확인
            if start_pos != -1:
                question = generate_question(picked['text'], keyword)
                print(f"Generated {i} question for '{keyword}(start pos_{start_pos})': {question}")
                
                # 데이터 저장
                data.append({
                    "title": picked['title'],
                    "context": picked['text'],
                    "document_id": picked['document_id'],
                    "keyword": keyword,
                    "start_pos": start_pos,
                    "question": question
                })
            else:
                print(f"Keyword '{keyword}' not found in text.")
    else:
        print("No keywords found or error occurred.")

# 새로운 데이터를 데이터프레임으로 변환
df_new = pd.DataFrame(data)

# 기존 데이터프레임과 합치기
df_result = pd.concat([df_existing, df_new], ignore_index=True)

# 데이터프레임을 CSV 파일로 저장 (덮어쓰기)
df_result.to_csv(csv_file_path, index=False)

# 데이터프레임 출력
print(df_result)


Existing file loaded.
Generated 0 question for '이평세(start pos_0)': UC 버클리 대학교에서 나노바이오포토닉스를 선도적으로 연구하고 있는 석좌교수는 누구인가?
Generated 1 question for 'B61(start pos_44)': 나토에 현재 배치된 핵무기의 종류는 무엇인가?
Generated 2 question for '웹사이트(start pos_0)': 인터넷 프로토콜 기반의 네트워크에서 도메인 이름이나 IP 주소를 통해 보이는 웹 페이지들의 의미 있는 묶음은 무엇인가?
Generated 3 question for '퀸(start pos_80)': 대한민국에서 2019년 4월 14일 기준 9,946,676명의 관객수를 기록하며 이례적인 흥행을 보인 밴드는 무엇인가?
Generated 4 question for '홀수번호(start pos_88)': 대한민국의 고속도로 노선 번호에서 남북방향은 어떤 번호가 부여되는가?
Generated 5 question for '지형(start pos_46)': 유닛의 방어가 결정되는 요소는 무엇인가?
Generated 6 question for '사회(start pos_101)': 사람은 본래 어디에서 나서 어디에서 자라고 어디에서 생존하는 존재인가?
Generated 7 question for '1034(start pos_434)': 2020년 1월 11일 기준으로 애니캐스트 어드레싱을 사용하여 루트 서버 인스턴스의 실제 수는 얼마인가?
Generated 8 question for '시의회(start pos_377)': 2012년 2월 8일, 포드 시장의 계획안을 뒤집은 기관은 무엇인가?
Generated 9 question for '앙리(start pos_18)': 프랑스 왕의 둘째 아들로 태어나 오를레앙 공작위를 받은 사람은 누구인가?
Generated 10 question for '렘브란트(start pos_246)': 17세기 네덜란드에서 시민의 주택을

In [93]:
from datasets import load_from_disk
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Arrow 파일 경로 설정
train_file_path = os.path.join(parent_dir, 'data', 'train_dataset')
test_file_path = os.path.join(parent_dir, 'data', 'test_dataset')
dataset = load_from_disk(train_file_path)
test_dataset=load_from_disk(test_file_path)

In [94]:
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
test_dataset=test_dataset['validation']

train_df=pd.DataFrame(train_dataset)
valid_df=pd.DataFrame(valid_dataset)
test_df=pd.DataFrame(test_dataset)

print(f"Train length : {len(train_df)}")
print(f"Valid length : {len(valid_df)}")
print(f"Test length : {len(test_df)}")

Train length : 3952
Valid length : 240
Test length : 600


In [95]:
train_df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [96]:

# __index_level_0__와 id 컬럼 삭제
train_df = train_df.drop(columns=["__index_level_0__", "id"])
valid_df = valid_df.drop(columns=["__index_level_0__", "id"])


In [97]:

# 어그멘테이션된 CSV 파일 로드
augmentation_csv_path = 'mrc_keywords_questions_data.csv'
augmented_df = pd.read_csv(augmentation_csv_path)

# 어그멘테이션 데이터를 기존 데이터프레임에 추가
# 필요한 컬럼 매핑이 제대로 되어 있는지 확인해야 함
# keyword를 answers로 변환: {'answer_start': [start_pos], 'text': [keyword]}
augmented_data = augmented_df[['context', 'keyword', 'question', 'document_id', 'start_pos']].rename(columns={
    'keyword': 'answers',  # 'answers'는 keyword를 text로 변환
    'question': 'question',
    'context': 'context',
    'document_id': 'document_id',
    'start_pos': 'start_pos'
})

# answers 컬럼을 {'answer_start': [start_pos], 'text': [keyword]} 형식으로 변경
augmented_data['answers'] = augmented_data.apply(lambda row: {'answer_start': [row['start_pos']], 'text': [row['answers']]}, axis=1)

# start_pos 컬럼은 이제 answers에 포함되므로 삭제
augmented_data = augmented_data.drop(columns=['start_pos'])


# 새로운 데이터를 기존 데이터셋에 추가 (train 데이터에 추가하는 예시)
train_augmented = pd.concat([train_df, augmented_data], ignore_index=True)

# 결과 저장 (필요 시 validation 및 test 데이터셋에도 추가 가능)
output_train_file = "augmented_train_dataset.csv"
train_augmented.to_csv(output_train_file, index=False)

# 결과 출력
print(f"Train data length after augmentation: {len(train_augmented)}")
print(train_augmented.head())


Train data length after augmentation: 5924
     title                                            context  \
0    미국 상원  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   
1   인사조직관리  '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...   
2      강희제  강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...   
3   금동삼존불감  불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...   
4  계사명 사리구  동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...   

                                  question  \
0         대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?   
1                   현대적 인사조직관리의 시발점이 된 책은?   
2           강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?   
3  11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?   
4               명문이 적힌 유물을 구성하는 그릇의 총 개수는?   

                                         answers  document_id  
0        {'answer_start': [235], 'text': ['하원']}        18293  
1  {'answer_start': [212], 'text': ['《경영의 실제》']}        51638  
2        {'answer_start': [510], 'text': ['백성']}         5028  
3        {'answer_start': [625], 

In [99]:
# Dataset으로 변환 (from pandas DataFrame)
train_dataset_augmented = Dataset.from_pandas(train_augmented)

# 동일하게 validation과 test도 처리 가능
valid_dataset_augmented = Dataset.from_pandas(valid_df)

# DatasetDict로 결합
dataset_dict = DatasetDict({
    "train": train_dataset_augmented,
    "validation": valid_dataset_augmented,
})

# Arrow 파일로 저장
output_dir = os.path.join(parent_dir, 'data', 'augmented_dataset2')
dataset_dict.save_to_disk(output_dir)

# 확인
print(f"Augmented dataset saved to {output_dir}")

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 59279.96 examples/s]

Augmented dataset saved to D:\boost-camp\07주_MRC\level2-mrc-nlp-04\data\augmented_dataset2
